In [4]:
import sys
sys.path.append('../..')
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import ta
import numpy as np
from utilities.data_manager import ExchangeDataManager
from utilities.bt_analysis import get_metrics, backtest_analysis
import nest_asyncio
nest_asyncio.apply()
from functools import reduce

In [228]:
class Strategy():
    def __init__(
        self,
        pair,
        type=["long"],
        params={},
    ):
        self.df_pair = None
        self.df = None
        self.pair = pair
        self.initial_wallet = 1000
        self.use_long = "long" in type
        self.use_short = "short" in type
        self.params = params
        self.result_df = None

    def get_pair_data(self, timeframe, start = 2050, end = 2050):
        exchange = ExchangeDataManager(
            exchange_name=exchange_name,
            path_download="./database/exchanges"
        )

        self.df_pair = exchange.load_data(self.pair, timeframe, start)

    def populate_indicators(self):
        params = self.params
        df = self.df_pair.copy()
        df.drop(
            columns=df.columns.difference(['open','high','low','close','volume']),
            inplace=True
        )

        # -- Populate indicators --
        df['fast_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["fast_ma"])
        df['slow_ma'] = ta.trend.sma_indicator(close=df["close"], window=params["slow_ma"])
        df['mrat'] = df['fast_ma'] / df['slow_ma']
        df['mean_mrat'] = ta.trend.sma_indicator(close=df['mrat'], window=params["mean_mrat_lenght"])
        df['stdev_mrat'] = df['mrat'].rolling(params["mean_mrat_lenght"]).std(ddof=0)
        df['open_long_signal'] = df['mean_mrat'].shift(1) - df['mrat'].shift(1) >= params['sigma_open'] * df['stdev_mrat'].shift(1)
        df['close_long_signal'] = df['mrat'].shift(1) - df['mean_mrat'].shift(1) >= params['sigma_close'] * df['stdev_mrat'].shift(1)

        df["is_liquidated"] = False
        df["order_open"] = False
        # Trading logic
        order_open = False
        current_order_number = 0
        open_price = 0
        quantity = 0
        trade_result = 0
        # Constants and Initialization
        initial_wallet = self.initial_wallet
        leverage = 2  # Fixed leverage
        maintenance_margin_percent = 0.004
        wallet = initial_wallet
        max_equity = initial_wallet  # To track the max equity before a new trade
        max_drawdown = 0

        for i in df.index:
            if df.loc[i, 'open_long_signal'] and not order_open:
                # Open a new order
                current_order_number += 1
                order_open = True
                open_price = df.loc[i, 'open']
                open_wallet = df.loc[i, 'wallet']
                quantity = (wallet / open_price) * leverage
                df.loc[i, 'order_number'] = current_order_number
                df.loc[i, 'order_open'] = order_open

            # Assign order_number to all rows of the current order
            if order_open:
                df.loc[i, 'order_number'] = current_order_number
                df.loc[i, 'order_open'] = order_open
                # Calculate hypothetical_wallet
                hypothetical_wallet = wallet + quantity * (df.loc[i, 'open'] - open_price)
                df.loc[i, 'hypothetical_wallet'] = hypothetical_wallet
                df.loc[i, 'quantity'] = quantity

                # Check for liquidation
                maintenance_margin = (wallet / leverage) * maintenance_margin_percent
                if hypothetical_wallet < maintenance_margin:
                    df.loc[i, 'is_liquidated'] = True
                    df.loc[i, 'trade_result'] = hypothetical_wallet - wallet
                    wallet = 0  # Update wallet with the loss
                    order_open = False  # Close the order

            # Close the order
            if df.loc[i, 'close_long_signal'] and order_open:
                trade_result = quantity * (df.loc[i, 'open'] - open_price)
                trade_result_perc = trade_result / wallet * 100
                wallet += trade_result  # Update wallet with the profit or loss
                order_open = False  # Close the order
                df.loc[i, 'trade_result'] = trade_result
                df.loc[i, 'trade_result_perc'] = trade_result_perc

            # Set wallet to current wallet value
            df.loc[i, 'wallet'] = wallet

        df["drawdown"] = (df["hypothetical_wallet"] - df["wallet"]) / df["wallet"] * 100

        return df


    def get_result_df(self):
        df = self.df
        final_wallet_amount = df.loc[df["order_open"] & df["close_long_signal"], "wallet"].tail(1)
        total_profit = final_wallet_amount  - self.initial_wallet
        total_profit_perc = total_profit / self.initial_wallet * 100
        total_trades = df["order_number"].max()
        avg_trade_profit_perc = df["trade_result_perc"].dropna().mean()
        avg_trade_profit = df["trade_result"].dropna().mean()
        max_drawdown = df["drawdown"].max()

        result_df = pd.DataFrame(
            {
                "params": str(self.params),
                "final_wallet_amount": final_wallet_amount,
                "total_profit": total_profit,
                "total_profit_perc": total_profit_perc,
                "total_trades": total_trades,
                "avg_trade_profit_perc": avg_trade_profit_perc,
                "avg_trade_profit": avg_trade_profit,
                "max_drawdown": max_drawdown,
            }
        )

        self.result_df = result_df

In [229]:
params = {
    "fast_ma": 6,
    "slow_ma": 60,
    "sigma_open": 2.3,
    "sigma_close": 2.3,
    "mean_mrat_lenght": 60
}

pair = "API3/USDT:USDT"
exchange_name = "binance"
tf = '15m'
oldest_pair = "API3/USDT:USDT"
start_date = "2023-01-01 00:00:00"

In [230]:
strat = Strategy(pair=pair, params=params)
strat.get_pair_data(timeframe=tf, start=start_date)
df1 = strat.populate_indicators()

In [223]:
df1.loc[df1["order_number"] == 4].iloc[[0,-1], :]

,open,high,low,close,volume,fast_ma,slow_ma,mrat,mean_mrat,stdev_mrat,...,close_long_signal,is_liquidated,order_open,wallet,order_number,hypothetical_wallet,quantity,trade_result,trade_result_perc,drawdown
date,,,,,,,,,,,,,,,,,,,,,
2024-02-16 07:40:00,4.3327,4.3381,4.3237,4.3312,93351.8,4.332117,4.423763,0.979283,0.989205,0.004213,...,False,False,True,1085.910036,4.0,1085.910036,250.631254,NaN,NaN,0.0
2024-02-16 09:50:00,4.4083,4.4091,4.3910,4.3970,117704.7,4.388267,4.378617,1.002204,0.988079,0.005572,...,True,False,True,1104.857759,4.0,1104.857759,250.631254,18.947723,1.74487,0.0


In [231]:
df1

,open,high,low,close,volume,fast_ma,slow_ma,mrat,mean_mrat,stdev_mrat,...,close_long_signal,is_liquidated,order_open,wallet,order_number,hypothetical_wallet,quantity,trade_result,trade_result_perc,drawdown
date,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,1.0120,1.0130,1.0020,1.0040,467066.5,NaN,NaN,NaN,NaN,NaN,...,False,False,False,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 00:15:00,1.0040,1.0080,1.0040,1.0070,203460.2,NaN,NaN,NaN,NaN,NaN,...,False,False,False,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 00:30:00,1.0070,1.0090,1.0030,1.0050,235463.6,NaN,NaN,NaN,NaN,NaN,...,False,False,False,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 00:45:00,1.0050,1.0100,1.0040,1.0080,185797.9,NaN,NaN,NaN,NaN,NaN,...,False,False,False,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 01:00:00,1.0090,1.0100,1.0070,1.0080,103208.8,NaN,NaN,NaN,NaN,NaN,...,False,False,False,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-18 22:45:00,3.1631,3.1828,3.1616,3.1828,30537.2,3.189267,3.198277,0.997183,0.978451,0.009994,...,False,False,True,16598.423692,147.0,15308.275779,10087.161162,NaN,NaN,-7.772713
2024-03-18 23:00:00,3.1831,3.2122,3.1720,3.2016,60735.7,3.187200,3.197543,0.996765,0.978655,0.010238,...,False,False,True,16598.423692,147.0,15510.019003,10087.161162,NaN,NaN,-6.557277
2024-03-18 23:15:00,3.2011,3.2191,3.1948,3.2133,42154.5,3.189050,3.197370,0.997398,0.978942,0.010515,...,False,False,True,16598.423692,147.0,15691.587904,10087.161162,NaN,NaN,-5.463385
